In [1]:
!pip install opencv-python numpy matplotlib torch torchvision ultralytics

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/216.1 MB ? eta -:--:--
   ---------------------------------------- 1.0/216.1 MB 6.3 MB/s eta 0:00:35
   ---------------------------------------- 1.8/216.1 MB 4.6 MB/s eta 0:00:47
   ---------------------------------------- 2.4/216.1 MB 4.5 MB/s eta 0:00:48
    --------------------------------------- 3.4/216.1 MB 4.2 MB/s eta 0:00:51
    --------------------------------------- 4.2/216.1 MB 4.1 MB/s eta 0:00:52
    --------------------------------------- 5.0/216.1 MB 3.9 MB/s eta 0:00:54
   - -------------------------------------- 5.5/216.1 MB 3.9 MB/s eta 0:00:54
   - -------------------------------------- 6.3/216.1 MB 3.9 MB/s eta 0:00:55
   - -------------------------------------- 7.1/216.1 MB 3.8 MB/s eta 0:00:56
   - -------------------------------------- 7.9/216.1 MB 3.8 MB/s eta 0:00:56
   - -------------------------------------- 8.7/216.1 MB 3.8 MB/s eta 0

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [2]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
from ultralytics import YOLO

Creating new Ultralytics Settings v0.0.6 file  
View Ultralytics Settings with 'yolo settings' or at 'C:\Users\HP\AppData\Roaming\Ultralytics\settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [3]:
model = YOLO("yolov8n.pt")  # Use yolov8s.pt for slightly better accuracy

100%|█████████████████████████████████████████████████████████████████████████████| 6.25M/6.25M [00:01<00:00, 4.13MB/s]


In [4]:
# Mapping COCO class IDs to vehicle labels
vehicle_classes = {2: "car", 3: "motorcycle", 7: "truck"}

def detect_vehicles(image_path, conf_threshold=0.5):
    results = model(image_path)[0]
    detections = []
    for box in results.boxes:
        cls_id = int(box.cls.item())
        conf = box.conf.item()
        if conf > conf_threshold and cls_id in vehicle_classes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            label = vehicle_classes[cls_id]
            detections.append((x1, y1, x2, y2, label, conf))
    return detections

def draw_boxes(image_path, detections):
    image = cv2.imread(image_path)
    for (x1, y1, x2, y2, label, conf) in detections:
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
        text = f"{label} {conf:.2f}"
        cv2.putText(image, text, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
    return image


In [5]:
from collections import Counter

input_dir = "data/test_images/"
output_dir = "output/processed_images/"
os.makedirs(output_dir, exist_ok=True)

summary_counts = Counter()

for file in os.listdir(input_dir):
    if file.endswith((".jpg", ".png")):
        img_path = os.path.join(input_dir, file)
        detections = detect_vehicles(img_path)
        
        for _, _, _, _, label, _ in detections:
            summary_counts[label] += 1
        
        # Draw and save output
        result_img = draw_boxes(img_path, detections)
        output_path = os.path.join(output_dir, file)
        cv2.imwrite(output_path, result_img)

        # Show one sample image inline
        img_rgb = cv2.cvtColor(result_img, cv2.COLOR_BGR2RGB)
        plt.imshow(img_rgb)
        plt.axis("off")
        plt.title(f"Processed: {file}")
        plt.show()


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'data/test_images/'